<a href="https://colab.research.google.com/github/tfarag/436---MRC/blob/main/MSCI__436_Movie_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install anvil-uplink

import anvil.server

anvil.server.connect("RCCG2I7RWLTPSG2H2NLG5GSI-6QMEGH2E54WX7FH3")

import pandas as pd
from google.colab import files
uploaded = files.upload()
df = pd.read_csv('filtered_filmtv_movies - ENG.csv')
#df = df.head(20000)
df.head()

df = df[['title', 'genre', 'country', 'actors', 'directors']]
df = df.fillna('') # Fill missing values with empty strings
df.head()

df['features'] = df['title'] + ' ' + df['genre'] + ' ' + df['country'] + ' ' + df['actors'] + ' ' + df['directors']

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english', min_df=20)
word_matrix = vectorizer.fit_transform(df['features'])
word_matrix.shape

from sklearn.metrics.pairwise import cosine_similarity

sim = cosine_similarity(word_matrix)

@anvil.server.callable
def get_recommendations(title, df, sim, count=10):
    # Get the row index of the specified title in the DataFrame
    index = df.index[df['title'].str.lower() == title.lower()]
    
    # Return an empty list if there is no entry for the specified title
    if (len(index) == 0):
        return []

    # Get the corresponding row in the similarity matrix
    similarities = list(enumerate(sim[index[0]]))
    
    # Sort the similarity scores in that row in descending order
    recommendations = sorted(similarities, key=lambda x: x[1], reverse=True)
    
    # Get the top n recommendations, ignoring the first entry in the list since
    # it corresponds to the title itself (and thus has a similarity of 1.0)
    top_recs = recommendations[1:count + 1]

    # Generate a list of titles from the indexes in top_recs
    titles = []

    for i in range(len(top_recs)):
        title = df.iloc[top_recs[i][0]]['title']
        titles.append(title)

    return titles

@anvil.server.callable
def get_recommendations_input(movie):
  titles = get_recommendations(movie, df, sim, count=10)
  return titles

anvil.server.wait_forever()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


Saving filtered_filmtv_movies - ENG.csv to filtered_filmtv_movies - ENG (1).csv
